In [1]:
import numpy as np
import pandas as pd 
import plotly.express as px
from get_similar_movies import similar_item_by_name

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [2]:
# !pip install dash-ag-grid

In [3]:
movies=pd.read_csv("movies.csv")
ratings=pd.read_csv("ratings.csv")
tags=pd.read_csv("tags.csv")
links=pd.read_csv("links.csv")

#movie ids are the same in all datasets

## preprocess

In [12]:
watch_count = ratings.groupby(['movieId']).agg({'userId': 'count'}).reset_index()
watch_count.columns=["movieId","watch count"]
watch_count.head()

,movieId,watch count
0,1,215
1,2,110
2,3,52
3,4,7
4,5,49


In [13]:
avg_rating=ratings.groupby("movieId").agg({"rating":"mean"}).reset_index()
avg_rating.columns=["movieId","avg ratings"]

In [14]:
available_ratings=ratings["movieId"].unique().tolist()

In [15]:
movies=movies[movies["movieId"].isin(available_ratings)]


In [16]:
movies_merged=pd.merge(movies,links[["tmdbId","imdbId","movieId"]], on=["movieId"]).merge(watch_count[["movieId","watch count"]],on=["movieId"])
movies_merged.shape

(9724, 6)

In [17]:
movies_metadata=pd.merge(movies_merged,avg_rating[["movieId","avg ratings"]], on=["movieId"])

In [18]:
movies_metadata.head()

,movieId,title,genres,tmdbId,imdbId,watch count,avg ratings
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,862.0,114709,215,3.920930
1,2,Jumanji (1995),Adventure|Children|Fantasy,8844.0,113497,110,3.431818
2,3,Grumpier Old Men (1995),Comedy|Romance,15602.0,113228,52,3.259615
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,31357.0,114885,7,2.357143
4,5,Father of the Bride Part II (1995),Comedy,11862.0,113041,49,3.071429


----------------------------------------------------------------------------------------------------

# Dashboard 

In [19]:
# !pip install dash
# !pip install jupyter_dash

In [20]:
import dash
from dash import Dash, dcc, html,dash_table
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import dash_ag_grid as dag
import time

In [21]:
app = dash.Dash(__name__,external_stylesheets=["assets\zephyr.min.css"])


In [22]:
#the tabs style when selected and unselected
tab_style = {
    "color": "black",
    'borderBottom': '1px solid #FCFCFC',
    'border-radius':'5px',
    "box-shadow": "1px 1px 0px F0A202",
    "border-left": "1px solid lightgrey !important",
   " border-right": "1px solid lightgrey !important",
    'border-radius':'5px',
    "border-top": "3px solid #e36209 !important",
     "background": '#F0A202',

    

}

tab_selected_style = {
    'fontWeight': 'bold',
   " box-shadow": "1px 1px 0px white",
    "border-left": "1px solid lightgrey !important",
   " border-right": "1px solid lightgrey !important",
    'border-radius':'5px',
    "border-top": "3px solid #e36209 !important",
    "font-color":"white"
    
    
}

In [23]:

tabs=dcc.Tabs(id='tabs-example-1', value='users', children=[
                            dcc.Tab(label='Users', value='users',style=tab_style, selected_style=tab_selected_style),
                            dcc.Tab(label='Movies', value='movies',style=tab_style, selected_style=tab_selected_style)
],content_style={"margin-top":"20px","margin-bottom":"-65px"},colors={"border-top": '#A4243B',})

#,colors={    "border-top": '#A4243B',    "primary": '#2E4057','#21897E'    "background": '#f9f9f9',} colors=border: '#d6d6d6',    primary: '#1975FA',    background: '#f9f9f9'

In [24]:
dashboard_header=dbc.CardHeader(
    [html.H1("🎞Tailored Films🎞",style={"textAlign":"center","margin-top":"20px","margin-bottom":"20px","color":"#2E4057"}),tabs],
style={ "border-bottom-color": "#FCFCFC","background-color":'#FECF72'}) 
 

In [25]:
content_card=dbc.Card([dashboard_header,dbc.CardBody([],id='tabs-example-content-1')], style={"margin-top":"20px"})#give it id to return the button based on the mode  

In [26]:
#modals and paginantion
recommendation_modal = dbc.Modal(
    
    id="modal-1", fullscreen=True,  size="xl",is_open=False
)


similarity_modal = dbc.Modal(
    id="modal-2", fullscreen=True,size="xl",is_open=False
)

### Recommendation

In [27]:
# Create dropdown menu options

dropdown = dcc.Dropdown(id='user-dropdown', options=ratings["userId"].unique(), value=ratings['userId'].unique()[0],style={"margin-top":"20px"})

button=dbc.Button("Get Recommendation for user", color="success",size="lg",id="recommendation_button",style={"margin-top":"20px","background-color":"05A8AA"})


# n_items=dbc.Input(id='n_page_modal-1', type='text', placeholder='Enter number of items per page')
# define the content for each modal as a dictionary
,




#add here input
user_content=dbc.Row([
    dropdown,dcc.Graph(id='graph-content',style={"margin-left":"40px","margin-top":"20px"})]),\
    dbc.Row([dbc.Col([button,recommendation_modal],width=3) ],justify="center")
#add filteration for genres and ratings ??

In [28]:
@app.callback(
    Output('graph-content', 'figure'),
    Input('user-dropdown', 'value')
)
def update_graph(value):
    
    tagged_users=tags.userId.unique()
    grouped_tags=tags.groupby(["userId","movieId"]).agg({'tag': lambda x: ', '.join(x)}).reset_index()

    filtered_graph=ratings[ratings["userId"]==value]

    merged_df = pd.merge(filtered_graph, movies[['movieId', 'title',"genres"]], on='movieId').sort_values('rating', ascending=False)
    if value in tagged_users.tolist():
        merged_df2=pd.merge(merged_df,grouped_tags[["movieId","tag","userId"]],on="userId")
        return px.funnel(merged_df2, x='rating',y='title',color="rating",hover_data=["tags"],title=f"user's {value} interaction history",template="zephyr").update_layout(width=1000, height=600)
   
    else :
        return px.funnel(merged_df, x='rating',y='title',color="rating",hover_data=["genres"],title=f"user's {value} interaction history").update_layout(width=1000, height=600)


In [29]:
movies["title"]

0                                Toy Story (1995)
1                                  Jumanji (1995)
2                         Grumpier Old Men (1995)
3                        Waiting to Exhale (1995)
4              Father of the Bride Part II (1995)
                          ...                    
9737    Black Butler: Book of the Atlantic (2017)
9738                 No Game No Life: Zero (2017)
9739                                 Flint (2017)
9740          Bungo Stray Dogs: Dead Apple (2018)
9741          Andrew Dice Clay: Dice Rules (1991)
Name: title, Length: 9724, dtype: object

In [30]:
movie_dropdown = dcc.Dropdown(id='movie-dropdown', options=movies["title"].unique(), value=movies["title"][0],style={"margin-top":"20px","margin-bottom":"20px"})
button=dbc.Button("Get Similar movies", color="success", className="me-1",size="lg",id="similarity_button",style={"margin-top":"20px","margin-bottom":"20px"}) #color="primary" return to dbc size="md"


#add here input too
movies_content=dbc.Row([movie_dropdown, html.Div(id='table-container')]),dbc.Row([dbc.Col([button,similarity_modal],width=3) ],justify="center")

In [31]:
@app.callback(
    Output('table-container', 'children'),
    Input('movie-dropdown', 'value')
)
def update_table(value):
    movies_filtered=movies_metadata[movies_metadata["title"]==value ]
    
    
    # define the CSS styles to customize the table
    table_style = {
        'backgroundColor': 'white',
        'border': '2px teal'
    }
    
    table = dbc.Table.from_dataframe(
    movies_filtered,
    bordered=True,
    hover=True,
#     style=table_style
    
  )
    
    
    return table
    

In [32]:
@app.callback(
    Output('tabs-example-content-1', 'children'),
    Input('tabs-example-1', 'value')
)
def render_content(tab):
    if tab == 'users':
        return user_content
    elif tab == 'movies':
        return movies_content


In [33]:
# open modal - user's modal
@app.callback(
    Output("modal-1", "is_open"),
    Input("recommendation_button", "n_clicks"),
    State("modal-1", "is_open"),
)
def toggle_modal_1(n_clicks, is_open):
    if n_clicks:
        return True
    else:
        return False

   

    ##movies callback
@app.callback(
    Output("modal-2", "is_open"),
    Input("similarity_button", "n_clicks"),
    State("modal-2", "is_open"),
)
def toggle_modal_2(n_clicks, is_open):
    if n_clicks:
        return True
    else:
        return False
    
    
#----------------------------------------------------------------------------------------

# update recommendation modal's content for user
@app.callback(
  Output("modal-1", "children"),
  Input('user-dropdown', 'value')
)
def update_modal_content(value):
    ##add model function 
    ##add daq table to be viewed with dcc.loading
  
    columnDefs = [
    {"field": "movieId"},
    {"field": "title"},
    {"field": "genres"},
    {"field": "tmdbId"},
    {"field": "imdbId"},
    {"field": "watch_count"},
    {"field": "avg_ratings"},
]
    
    input_field=dcc.Input(id="input-page-size", type="number", min=1, max=len(movies_metadata), value=5)
    
    table=dag.AgGrid(
            id="grid-page-size",
            columnDefs=columnDefs,
        
            #where to change
            rowData=movies_metadata.to_dict("records"),
            columnSize="sizeToFit",
            defaultColDef={"resizable": True,"suppressMovable": True},
            dashGridOptions={"pagination": True},
            suppressDragLeaveHidesColumns=True,
            className="ag-theme-material"
        )
    
    #can be removed
    loading_table = dcc.Loading(
    type="dot",
    id="loading-table",
    children=[table]
)

    
    modal_1_content =\
    dbc.ModalHeader(dbc.ModalTitle(f"Movie Recommendation for user's {value}"),style={'textAlign':'center'}),html.Hr(),dbc.ModalBody([
        dbc.Row([ 
            dbc.Col([html.P("Please enter number of recommendations")],width=4),dbc.Col([input_field],width=3,style={"margin-left":"-35px"})]),
        loading_table,
        
       
    ],id="recommendation_modal")

    return modal_1_content



In [34]:
#update number of columns
@app.callback(
    Output("grid-page-size", "dashGridOptions"),
    Input("input-page-size", "value"),
    State("grid-page-size", "dashGridOptions"),
)
def update_page_size(page_size, grid_options):
    page_size = 1 if page_size is None else page_size
    grid_options["paginationPageSize"] = page_size
    return grid_options

In [35]:
# @app.callback(Output("loading-table", "children"), Input("grid-page-size", "value"))
# def input_triggers_spinner(value):
#     time.sleep(3)
#     table=dag.AgGrid(
#             id="grid-page-size",
#             columnDefs=columnDefs,
#             rowData=movies_metadata.to_dict("records"),
#             columnSize="sizeToFit",
#             defaultColDef={"resizable": True,"suppressMovableColumns": True},
#             dashGridOptions={"pagination": True,"paginationPageSize":value},
#             suppressDragLeaveHidesColumns=True
#     )
#     return table
# #     return value

#pause currently

In [36]:
#update similarity's modal's content
@app.callback(
  Output("modal-2", "children"),
  Input('movie-dropdown', 'value')
)
def update_modal_content2(value):
    ##add model function 
    ##add daq table to be viewed with dcc.loading
  
    columnDefs = [
    {"field": "movieId"},
    {"field": "title"},
    {"field": "genres"},
    {"field": "tmdbId"},
    {"field": "imdbId"},
    {"field": "similarity score"},
    {"field": "avg ratings"},
    {"field": "watch count"},
]
    
    input_field=dcc.Input(id="input-movie-size", type="number", min=1, max=len(movies_metadata), value=5)
    similar_films=similar_item_by_name(value)
    #add hovering effect on titles
    table=dag.AgGrid(
            id="grid-movie-size",
            columnDefs=columnDefs,
            #where to change
            rowData=similar_films.to_dict("records"),
            columnSize="sizeToFit",
            defaultColDef={"resizable": True,"suppressMovable": True},
            dashGridOptions={"pagination": True},
            suppressDragLeaveHidesColumns=True,
            className="ag-theme-material"
            
        )
    
    loading_table = dcc.Loading(
    type="dot",
    id="loading-items",
        
    children=[table]
)

    
    modal_2_content =\
    dbc.ModalHeader(dbc.ModalTitle(f"Similar movies for for: {value}"),style={'textAlign':'center'}),html.Hr(),dbc.ModalBody([
        dbc.Row([ 
        dbc.Col([html.P("Please enter number of recommendations")],width=4),dbc.Col([input_field],width=3,style={"margin-left":"-35px"})
        ]),
        loading_table,
       
    ],id="similarity_modal")

    return modal_2_content



In [37]:
# @app.callback(Output("loading-items", "children"), Input('movie-dropdown', 'value'))
# def input_triggers_spinner(value):
#     columnDefs = [
#     {"field": "movieId"},
#     {"field": "title"},
#     {"field": "genres"},
#     {"field": "tmdbId"},
#     {"field": "imdbId"},
#     {"field": "similarity score"},
#     {"field": "avg ratings"},
#     {"field": "watch count"},
# ]
  
#     similar_films=similar_item_by_name(value)
#     #add hovering effect on titles
#     table=dag.AgGrid(
#             id="grid-movie-size",
#             columnDefs=columnDefs,
#             #where to change
#             rowData=similar_films.to_dict("records"),
#             columnSize="sizeToFit",
#             defaultColDef={"resizable": True,"suppressMovableColumns": True},
#             dashGridOptions={"pagination": True},
#             suppressDragLeaveHidesColumns=True
            
#         )
#     return table
# #     return value

# # pause currently

In [38]:
#update number of columns
@app.callback(
    Output("grid-movie-size", "dashGridOptions"),
    Input("input-movie-size", "value"),
    State("grid-movie-size", "dashGridOptions"),
)
def update_page_size2(page_size, grid_options):
    page_size = 1 if page_size is None else page_size
    grid_options["paginationPageSize"] = page_size
    return grid_options

In [39]:
footer = html.Footer(
       
            html.P("ITI © Salma Kishk 🤞 Mariam Gamal 🤞 Yasmeen Nour",style={'textAlign':'center',"margin-top":"50px","font-color":"#808080"})
)
    


In [40]:
app.layout = dbc.Container([
                            content_card,
                            footer
                          ])

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False,port=8050)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [24/Jun/2023 23:57:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2023 23:57:47] "GET /assets/zephyr.min.css HTTP/1.1" 304 -
127.0.0.1 - - [24/Jun/2023 23:57:47] "GET /assets/_bootswatch.scss?m=1686863302.6586335 HTTP/1.1" 304 -
127.0.0.1 - - [24/Jun/2023 23:57:47] "GET /assets/zephyr.min.css?m=1686866740.2710085 HTTP/1.1" 304 -
127.0.0.1 - - [24/Jun/2023 23:57:48] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2023 23:57:48] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2023 23:57:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2023 23:57:52] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2023 23:57:53] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [24/Jun/2023 23:57:53] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [24/Jun/2023 23:57:53] "GET /_dash-component-suites